In [1]:
import torch
import numpy as np

In [6]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Параметры задачи
D = 1                # Коэффициент диффузии
lambda_ = 1         # Интенсивность источника
p = 10               # Порог для функции F
mu = 3               # Амплитуда начального возмущения
T_delay = 10.0         # Временная задержка
L = 2 * np.pi         # Пространственный период
Nx = 512              # Число пространственных точек
t_max = 40.0          # Время моделирования
dt = 0.0005           # Шаг по времени
save_interval = 1     # Интервал сохранения результатов

# Пространственная сетка
x = torch.linspace(0, L, Nx, device=device)
dx = L / (Nx - 1)

# Подготовка волновых чисел для FFT
k = 2 * torch.pi * torch.fft.fftfreq(Nx, d=dx).to(device)
k2 = k**2

Using device: cuda


In [7]:
# Историческая память для задержки
history_steps = int(T_delay / dt)
u_history = torch.ones((history_steps, Nx), device=device) * (p + mu)

# Начальное условие
phi = p + mu * (torch.sin((x+torch.pi)/2.0)**2).to(device)
u_current = phi.clone()


In [8]:
# Сохранение результатов
save_steps = int(t_max / (dt * save_interval))
solutions = torch.zeros((save_steps, Nx), device=device)
times = np.zeros(save_steps)
critical_times = []  # Для хранения моментов касания границы p
save_idx = 0

# Основной цикл моделирования
for step in range(int(t_max / dt)):
    # Получение запаздывающего члена
    u_delayed = u_history[0]
    
    # Вычисление функции F
    F = torch.where(torch.abs(u_delayed) >= p, 
                   torch.zeros_like(u_delayed), 
                   u_delayed)
    
    # Преобразование Фурье
    F_hat = torch.fft.fft(F)
    u_hat = torch.fft.fft(u_current)
    
    # IMEX схема
    denominator = 1 + dt * (D * k2 + 1)
    numerator = u_hat + dt * lambda_ * F_hat
    u_hat_next = numerator / denominator
    
    # Обратное преобразование Фурье
    u_next = torch.fft.ifft(u_hat_next).real
    
    # Обновление истории
    u_history = torch.cat([u_history[1:], u_current.unsqueeze(0)])
    
    # Сохранение результатов и проверка на касание границы p
    if step % save_interval == 0:
        solutions[save_idx] = u_current
        times[save_idx] = step * dt
        
        # Проверка, касается ли решение границы p
        min_val = torch.min(u_current).item()
        if min_val <= p:
            critical_times.append(step * dt)
        
        save_idx += 1
    
    # Обновление текущего состояния
    u_current = u_next

In [9]:
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, VBox, HBox, Label, Button, Output, Tab
import ipywidgets as widgets
import os
import re
from datetime import datetime
import numpy as np

# Преобразуем результаты в numpy
x_np = x.cpu().numpy()
solutions_np = solutions.cpu().numpy()

# Группируем критические моменты и выбираем первый из каждой группы
def group_critical_times(times, time_window=0.5):
    if not times:
        return []
    
    times.sort()
    groups = []
    current_group = [times[0]]
    
    for t in times[1:]:
        if t - current_group[-1] <= time_window:
            current_group.append(t)
        else:
            groups.append(current_group)
            current_group = [t]
    
    groups.append(current_group)
    return [group[0] for group in groups]

key_critical_times = group_critical_times(critical_times)
key_times = [0.0] + key_critical_times + [times[-1]]

# Создаем вкладки
tab1 = Output()  # Пространственный срез
tab2 = Output()  # Временной срез
tab3 = Output()  # 3D визуализация

tabs = Tab(children=[tab1, tab2, tab3])
tabs.set_title(0, 'Пространственный срез')
tabs.set_title(1, 'Временной срез')
tabs.set_title(2, 'Тепловая карта')

# Глобальные переменные
current_fig = None
current_time = 0.0

# 1. Функция для пространственного среза
def plot_spatial_slice(t=0.0):
    global current_fig, current_time
    
    idx = np.abs(times - t).argmin()
    actual_time = times[idx]
    current_time = actual_time
    u_slice = solutions_np[idx]
    
    min_val = np.min(u_slice)
    max_val = np.max(u_slice)
    touching_p = min_val <= p
    
    y_range = [min(min_val, p) - 0.1, min(max_val, p + mu) + 0.1]
    
    with tab1:
        tab1.clear_output(wait=True)
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=x_np, 
            y=u_slice,
            mode='lines',
            line=dict(color='blue', width=2),
            name=f'u(x) при t={actual_time:.2f}'
        ))
        
        if touching_p:
            min_idx = np.argmin(u_slice)
            fig.add_trace(go.Scatter(
                x=[x_np[min_idx]],
                y=[u_slice[min_idx]],
                mode='markers',
                marker=dict(color='red', size=10),
                name=f'Касание p: {u_slice[min_idx]:.2f}'
            ))
        
        fig.add_hline(
            y=p, 
            line_dash="dash", 
            line_color="green",
            annotation_text="p", 
            annotation_position="bottom right"
        )
        
        fig.add_hline(
            y=p+mu, 
            line_dash="dash", 
            line_color="red",
            annotation_text="p+mu", 
            annotation_position="top right"
        )
        
        title = f'Срез решения при t={actual_time:.2f}'
        if touching_p:
            title += f' - КАСАНИЕ p (min={min_val:.2f})'
        
        fig.update_layout(
            title=title,
            xaxis_title='Пространство x',
            yaxis_title='Значение u(x,t)',
            height=500,
            yaxis_range=y_range,
            showlegend=True
        )
        
        fig.update_xaxes(
            tickvals=[0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi],
            ticktext=['0', 'π/2', 'π', '3π/2', '2π'],
            tickfont=dict(size=12),
            title_font=dict(size=14)
        )
        
        current_fig = fig
        fig.show()

# 2. Функция для временного среза
def plot_time_slice(x0=0.0):
    global current_fig
    with tab2:
        tab2.clear_output(wait=True)
        
        # Находим ближайшую точку в пространстве
        x_idx = np.abs(x_np - x0).argmin()
        actual_x = x_np[x_idx]
        u_time_slice = solutions_np[:, x_idx]
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=times,
            y=u_time_slice,
            mode='lines',
            line=dict(color='purple', width=2),
            name=f'u(t) при x={actual_x:.2f}'
        ))
        
        # Добавляем критические моменты времени
        for t in key_critical_times:
            idx_t = np.abs(times - t).argmin()
            fig.add_trace(go.Scatter(
                x=[times[idx_t]],
                y=[u_time_slice[idx_t]],
                mode='markers',
                marker=dict(color='orange', size=8),
                name=f'Критический момент: {t:.2f}'
            ))
        
        fig.add_hline(
            y=p, 
            line_dash="dash", 
            line_color="green",
            annotation_text="p", 
            annotation_position="bottom right"
        )
        
        fig.update_layout(
            title=f'Эволюция решения в точке x={actual_x:.2f}',
            xaxis_title='Время t',
            yaxis_title='Значение u(x,t)',
            height=500,
            showlegend=True
        )

        current_fig = fig
        fig.show()

# 3. Функция для тепловой карты (транспонированная)
def plot_heatmap():
    global current_fig
    with tab3:
        tab3.clear_output(wait=True)
        
        # Прореживаем данные для оптимизации
        time_step = max(1, len(times) // 500)
        x_step = max(1, len(x_np) // 500)
        
        # Транспонируем данные и меняем оси местами
        fig = go.Figure(data=go.Heatmap(
            z=solutions_np[::time_step, ::x_step].T,  # ТРАНСПОНИРОВАННЫЕ ДАННЫЕ
            x=times[::time_step],                     # Время на оси X
            y=x_np[::x_step],                         # Пространство на оси Y
            colorscale='Viridis',
            hoverongaps=False,
            colorbar=dict(title='u(x,t)')
        ))
        
        # Добавляем маркеры для критических моментов времени (вертикальные линии)
        for t in key_critical_times:
            fig.add_vline(
                x=t,
                line_dash="dash",
                line_color="yellow",
                opacity=0.7,
                annotation_text=f"{t:.2f}",
                annotation_position="top right",
                annotation_bgcolor="yellow"
            )
        
        fig.update_layout(
            title='Тепловая карта решения u(x,t)',
            xaxis_title='Время t',                # Теперь время на оси X
            yaxis_title='Пространство x',          # Пространство на оси Y
            height=700,
            width=1000
        )
        
        fig.update_yaxes(
            tickvals=[0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi],
            ticktext=['0', 'π/2', 'π', '3π/2', '2π'],
            tickfont=dict(size=10)
        )
        
        current_fig = fig
        fig.show()

# Кнопки для переключения между ключевыми моментами
buttons = []
for i, t in enumerate(key_times):
    btn = Button(description=f"t={t:.4f}", layout={'width': '120px'})
    btn.time_value = t
    buttons.append(btn)

# Слайдер для выбора точки в пространстве
x_slider = FloatSlider(
    value=0.0,
    min=0,
    max=L,
    step=dx,
    description='x:',
    continuous_update=False,
    layout={'width': '80%'}
)

# Кнопка для сохранения графиков
save_button = Button(
    description="Сохранить график в HTML",
    button_style='success',
    icon='save'
)

# Область для вывода сообщений о сохранении
save_output = Output()

# Привязка функций
def on_spatial_button_click(b):
    plot_spatial_slice(b.time_value)

def on_time_slider_change(change):
    plot_time_slice(change['new'])

for btn in buttons:
    btn.on_click(on_spatial_button_click)

x_slider.observe(on_time_slider_change, names='value')

# Функция сохранения
def save_html(b):
    global current_fig
    if current_fig is not None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"graph_t_{current_time:.2f}_{timestamp}.html"
        current_fig.write_html(filename)
        
        save_output.clear_output()
        with save_output:
            print(f"График сохранен в файл: {filename}")
            print(f"Полный путь: {os.path.abspath(filename)}")
    else:
        save_output.clear_output()
        with save_output:
            print("Нет активного графика для сохранения")

save_button.on_click(save_html)

# Информационная панель
if key_critical_times:
    critical_info = Label(value=f"Ключевые критические моменты: {', '.join(f'{t:.2f}' for t in key_critical_times)}")
else:
    critical_info = Label(value="Критические моменты не обнаружены")

# Отображение всех элементов
display(VBox([
    critical_info,
    HBox(buttons),
    HBox([save_button]),
    tabs,
    widgets.HBox([Label("Выберите точку в пространстве:"), x_slider]),
    save_output
]))

# Инициализация графиков
plot_spatial_slice(0.0)
plot_time_slice(0.0)
plot_heatmap()